In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração do estilo dos gráficos
sns.set(style="whitegrid")



# 🧹 **Tratamento Completo dos Dados de Diagnóstico de Câncer**
🔍 *Garantindo qualidade antes da análise*

---

## **📌 Introdução**
Antes de realizarmos qualquer análise, precisamos garantir que os dados estão **limpos e confiáveis**. Dados mal estruturados podem levar a conclusões erradas, afetando pesquisas e decisões de saúde pública.

Neste notebook, vamos tratar os dados da **base de diagnósticos de câncer** do **Registro de Câncer de Base Populacional (RCBP) de Poços de Caldas**.  

**Etapas do tratamento de dados:**  

✅ Verificação e remoção de **valores ausentes** em colunas essenciais.  
✅ Tratamento de **datas inválidas ou inconsistentes**.  
✅ Identificação e correção de **outliers temporais**.  
✅ Remoção ou ajuste de **anos incompletos** para evitar distorções na análise.  

Após o tratamento, reanalisaremos a **taxa de crescimento anual de diagnósticos** de forma mais confiável. 🚀  


In [ ]:

# Carregar o arquivo
file_path = "/mnt/data/base_nao_identificada_3702.csv"

# Ler o dataset
df = pd.read_csv(file_path, encoding="ISO-8859-1", delimiter=";")

# Exibir informações iniciais
df.info()
df.head()



## 📊 **Valores Ausentes**
Vamos verificar se existem valores ausentes nas colunas essenciais, como **data de diagnóstico**.


In [ ]:

# Contagem de valores ausentes
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values



### 🔧 **Tratando valores ausentes**
Para evitar distorções na análise, vamos:

1. **Remover registros sem data de diagnóstico**, pois não conseguimos analisá-los.  
2. **Manter outros valores ausentes**, pois podem ser irrelevantes para esta análise.  


In [ ]:

# Removendo registros sem data de diagnóstico
df = df.dropna(subset=["Data de Diagnostico"])

# Verificando novamente
df.isnull().sum()



## 🗓 **Verificando Datas Inválidas**
Datas erradas podem prejudicar a análise. Vamos garantir que:

✅ Todas as datas estejam no formato correto.  
✅ Não existam diagnósticos em **anos futuros**.  
✅ Não existam diagnósticos em **anos muito antigos** que não fazem sentido no contexto do estudo.  


In [ ]:

# Converter a coluna de Data de Diagnóstico para formato de data
df["Data de Diagnostico"] = pd.to_datetime(df["Data de Diagnostico"], format="%d/%m/%Y", errors="coerce")

# Criar a coluna do ano de diagnóstico
df["Ano do Diagnostico"] = df["Data de Diagnostico"].dt.year

# Verificar valores mínimos e máximos de ano
print("Ano mais antigo registrado:", df["Ano do Diagnostico"].min())
print("Ano mais recente registrado:", df["Ano do Diagnostico"].max())



### 🚨 **Corrigindo Datas Futuras ou Erradas**
Se houver registros em anos futuros ou muito antigos, vamos removê-los.


In [ ]:

# Remover anos que não fazem sentido (exemplo: acima do ano atual)
ano_atual = pd.Timestamp.today().year
df = df[(df["Ano do Diagnostico"] <= ano_atual) & (df["Ano do Diagnostico"] >= 1900)]

# Verificar novamente
df["Ano do Diagnostico"].describe()



## 📉 **Verificando Anos com Poucos Registros**
Se houver anos com poucos registros, pode ser um problema de coleta de dados e devemos decidir se os incluímos na análise.


In [ ]:

# Contar diagnósticos por ano
diagnosticos_por_ano = df["Ano do Diagnostico"].value_counts().sort_index()

# Visualizar a distribuição
plt.figure(figsize=(10,5))
sns.barplot(x=diagnosticos_por_ano.index, y=diagnosticos_por_ano.values, palette="Blues_r")
plt.xticks(rotation=45)
plt.title("Número de Diagnósticos por Ano", fontsize=14)
plt.xlabel("Ano", fontsize=12)
plt.ylabel("Número de Diagnósticos", fontsize=12)
plt.show()



### ✂️ **Removendo Anos com Registros Insuficientes**
Se houver anos com **poucos diagnósticos**, podemos removê-los para evitar distorções.


In [ ]:

# Definir um limite mínimo (exemplo: anos com menos de 10 casos podem ser removidos)
limite_minimo = 10
anos_validos = diagnosticos_por_ano[diagnosticos_por_ano >= limite_minimo].index

# Filtrar apenas os anos válidos
df = df[df["Ano do Diagnostico"].isin(anos_validos)]

# Verificar novamente
df["Ano do Diagnostico"].value_counts().sort_index()



## ✅ **Conclusão do Tratamento de Dados**
Após a limpeza, nossos dados estão prontos para análise! 🚀

**Modificações realizadas:**  
✔ Removemos registros **sem data de diagnóstico**.  
✔ Eliminamos **datas inválidas e diagnósticos em anos futuros**.  
✔ Identificamos e **removemos anos com registros insuficientes**.  

Agora podemos seguir para a análise da **taxa de crescimento anual de diagnósticos de câncer** com mais confiança!  
